In [1]:
# load in packages required for analysis.

%matplotlib qt
import os.path

from refnx.dataset import ReflectDataset
from refnx.analysis import Transform, CurveFitter, Objective, GlobalObjective, Parameter
from refnx.reflect import SLD, ReflectModel, MixedReflectModel
from MixedLayerSlab import MixedMagSlabs2 #load in custom model for magnetic layers in non-polarised instrument.
import refnx

import dynesty

import scipy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#print what versions we are using.

print('refnx: %s\nscipy: %s\nnumpy: %s' % (refnx.version.version, scipy.version.version, np.version.version))

refnx: 0.1.17
scipy: 1.5.2
numpy: 1.19.2


In [3]:
#load data from .txt files - this can be done smarter but works as a test.

file_path_hdod = '61112_14.txt'
file_path_ddod = '61143_45.txt'

#Define datasets with data loaded

P1hdod = ReflectDataset(file_path_hdod)
P1ddod = ReflectDataset(file_path_ddod)

In [4]:
#now define some SLDs we will use...

#LAYERS#
Si = SLD(2.07, name='Si')
SiO2 = SLD(3.47, name='SiO2')
Fe = SLD(8.02, name='Fe')
FeOx = SLD(7.0, name='FeOx')

#SOLVENTS#
hdod_SLD = SLD(-0.462, name='hdod1')
ddod_SLD = SLD(6.7, name='ddod1')

In [5]:
"""
Here we define the material layers at the interface.
"""

#Create slabs for model

##SiO2##

SiO2_lay = SiO2(15, 3)

SiO2_lay.thick.setp(vary=True, bounds=(1, 25))

##Fe##
# share the same Fe thickness, SiO2-Fe roughness - we specify variation of parameters here.
SiO2_Fe_R = Parameter(4, 'SiO2_Fe_R', vary=True, bounds=(1, 10))
Fe.real.setp(vary=True, bounds=(7.5, 8.1))
Fe_thick = Parameter(190, 'Fe_t', vary=True, bounds=(170, 210))
Fe_magmom = Parameter(2.1, 'Fe_magmom', vary=True, bounds=(1.9, 2.2))
Fe_ScattLen = Parameter(0.0000945, 'Fe_SL', vary=False)

FeUp_lay = MixedMagSlabs2.USM_Slab(Fe_thick, Fe, Fe_magmom, Fe_ScattLen, SiO2_Fe_R)
FeDown_lay = MixedMagSlabs2.DSM_Slab(Fe_thick, Fe, Fe_magmom, Fe_ScattLen, SiO2_Fe_R)

##FeOx##
Fe_FeOx_R = Parameter(5, 'FeOx_Fe_R', vary=True, bounds=(1, 15))
FeOx.real.setp(vary=True, bounds=(5.0, 7.2))
FeOx_thick = Parameter(30, 'FeOx_t', vary=True, bounds=(20, 40))
FeOx_m = Parameter(0.5, 'FeOx_m', vary=True, bounds=(0, 1.3))

FeOxUp_lay = MixedMagSlabs2.USM_nomagmom_Slab(FeOx_thick, FeOx, FeOx_m, Fe_FeOx_R)
FeOxDown_lay = MixedMagSlabs2.DSM_nomagmom_Slab(FeOx_thick, FeOx, FeOx_m, Fe_FeOx_R)

FeOx_R = Parameter(6, 'FeOx_R', vary=True, bounds=(1, 15))

### SOLVENT PARAMS FOR BARE_LAYER SOLVENT###

hdod_bare_lay = hdod_SLD(0, FeOx_R)
ddod_bare_lay = ddod_SLD(0, FeOx_R)

ddod_bare_lay.sld.real.setp(vary=True, bounds=(5, 6.703))
hdod_bare_lay.sld.real.setp(vary=False) #we could fit using a gaussian prior made from pre-determined density measurements.
#however, probably good enough just to keep a constant at -0.462 x 10-6 \AA-2.

In [6]:
P1_hdod_U_bare = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | hdod_bare_lay
P1_hdod_D_bare = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | hdod_bare_lay

P1_ddod_U_bare = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | ddod_bare_lay
P1_ddod_D_bare = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | ddod_bare_lay

In [7]:
#now define the models

#these are the scale and background parameters
P1hdod_M_int = Parameter(0.5, 'P1hdod_M_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
P1ddod_M_int = Parameter(0.5, 'P1ddod_M_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
P1hdod_M_bkg = Parameter(1e-6, 'P1hdod_M_bkg', vary=True, bounds=(1e-7, 2e-5))
P1ddod_M_bkg = Parameter(1e-6, 'P1ddod_M_bkg', vary=True, bounds=(1e-7, 2e-5))

#define bare interface objective & global objective.
P1hdod_M_bare = MixedReflectModel((P1_hdod_U_bare, P1_hdod_D_bare), scales=(P1hdod_M_int, P1hdod_M_int), bkg=P1hdod_M_bkg, dq=4.7096)
P1ddod_M_bare = MixedReflectModel((P1_ddod_U_bare, P1_ddod_D_bare), scales=(P1ddod_M_int, P1ddod_M_int), bkg=P1ddod_M_bkg, dq=4.7096)
P1hdod_bare_obj = Objective(P1hdod_M_bare, P1hdod, transform=Transform('logY'))
P1ddod_bare_obj = Objective(P1ddod_M_bare, P1ddod, transform=Transform('logY'))
bare_glob_obj = GlobalObjective([P1hdod_bare_obj, P1ddod_bare_obj])

In [8]:
### BE SENSIBLE AND HAVE A LOOK AT YOUR DATA BEFORE NS ###

P1hdod_bare_obj.plot()
P1ddod_bare_obj.plot()

### ALSO HAVE A LOOK AT THE SLD TO CHECK ALL LOOKS FINE ###

# plt.plot(*P1_hdod_U_bare.sld_profile())
# plt.plot(*P1_hdod_D_bare.sld_profile())
# plt.ylabel('SLD /$10^{-6} \AA^{-2}$')
# plt.xlabel('distance / $\AA$');

### makes sense to look at the list of varying parameters to ensure no mistakes ###
print(bare_glob_obj.varying_parameters().names())

['P1hdod_M_int', 'P1hdod_M_bkg', 'SiO2 - thick', 'Fe_t', 'Fe - sld', 'Fe_magmom', 'SiO2_Fe_R', 'FeOx_t', 'FeOx - sld', 'FeOx_m', 'FeOx_Fe_R', 'FeOx_R', 'P1ddod_M_int', 'P1ddod_M_bkg', 'ddod1 - sld']


In [9]:
### initial fit to check the model works how we wish. ###
fitter = CurveFitter(bare_glob_obj)
fitter.fit('differential_evolution'); #initial fit. lets check the model is how we want.
print(bare_glob_obj)

133it [03:51,  1.74s/it]


_______________________________________________________________________________

--Global Objective--
________________________________________________________________________________
Objective - 2068013509832
Dataset = 61112_14
datapoints = 186
chi2 = 772.5678290586268
Weighted = True
Transform = Transform('logY')
________________________________________________________________________________
Parameters:       ''       
[Parameters(data=[Parameters(data=[Parameter(value=0.45116777962732135, name='P1hdod_M_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None), Parameter(value=0.45116777962732135, name='P1hdod_M_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None)], name='scale factors'), Parameter(value=5.155824342011186e-06, name='P1hdod_M_bkg', vary=True, bounds=Interval(lb=1e-07, ub=2e-05), constraint=None), Parameter(value=4.7096, name='dq - resolution', vary=False, bounds=Interval(lb=-np.inf, ub=np.inf), constraint=None)], name='instrument parameters')]


In [10]:
#check acceptable fits and SLD profiles...
P1hdod_bare_obj.plot()
P1ddod_bare_obj.plot()

# plt.plot(*P1_ddod_U_bare.sld_profile())
# plt.plot(*P1_ddod_D_bare.sld_profile())
# plt.ylabel('SLD /$10^{-6} \AA^{-2}$')
# plt.xlabel('distance / $\AA$');

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:>)

In [11]:
### Nested Sampling ### Now we do the nested sampling!
nested_sampler = dynesty.NestedSampler(bare_glob_obj.logl, bare_glob_obj.prior_transform, ndim=len(bare_glob_obj.varying_parameters()))
nested_sampler.run_nested()

30272it [1:31:20,  5.52it/s, +500 | bound: 498 | nc: 1 | ncall: 729888 | eff(%):  4.216 | loglstar:   -inf < -117.027 <    inf | logz: -177.010 +/-  0.495 | dlogz:  0.001 >  0.509]


In [12]:
ns_results_bare = nested_sampler.results

from dynesty import plotting as dyplot

#print(ns_results)

# Plot a summary of the run.
rfig, raxes = dyplot.runplot(ns_results_bare)

# Plot traces and 1-D marginalized posteriors.
#tfig, taxes = dyplot.traceplot(ns_results_bare)

# Plot the 2-D marginalized posteriors.
#cfig, caxes = dyplot.cornerplot(ns_results_bare, labels=bare_glob_obj.varying_parameters().names())

print(ns_results_bare.logz[-1], ns_results_bare.logzerr[-1])

-177.01033670662588 0.4946925113693943
